# PyMR vs TwoSampleMR Validation

This notebook validates PyMR results against the established R package TwoSampleMR.

We use a well-studied example: the causal effect of Body Mass Index (BMI) on Type 2 Diabetes (T2D).

## Comparison Metrics

We compare the following estimates:
- **IVW** (Inverse Variance Weighted): Beta, SE, p-value
- **Weighted Median**: Beta, SE, p-value
- **MR-Egger**: Slope, intercept, SE values, p-values
- **Heterogeneity Statistics**: Cochran's Q, degrees of freedom, I²

## Data

We use simulated data that mimics real GWAS summary statistics.

In [ ]:
import numpy as np
import pandas as pd
from pymr.methods import ivw, weighted_median, mr_egger
from pymr.sensitivity import cochrans_q

## Simulated Example Dataset

We create a dataset with:
- 20 SNPs (genetic instruments)
- True causal effect of 0.4 (BMI → T2D)
- Realistic effect sizes and standard errors based on typical GWAS

In [ ]:
# Set seed for reproducibility
np.random.seed(42)

# Number of SNPs
n_snps = 20

# SNP identifiers
snp_ids = [f"rs{1000000 + i}" for i in range(n_snps)]

# Exposure effects (BMI) - typical range for genome-wide significant SNPs
beta_exp = np.random.uniform(0.05, 0.15, n_snps)
se_exp = np.random.uniform(0.01, 0.02, n_snps)

# True causal effect
true_causal_effect = 0.4

# Outcome effects (T2D) - generated from true causal effect plus noise
beta_out = true_causal_effect * beta_exp + np.random.normal(0, 0.01, n_snps)
se_out = np.random.uniform(0.015, 0.03, n_snps)

# Create DataFrame
data = pd.DataFrame({
    'SNP': snp_ids,
    'beta_exp': beta_exp,
    'se_exp': se_exp,
    'beta_out': beta_out,
    'se_out': se_out
})

print("Dataset Summary:")
print(f"Number of SNPs: {n_snps}")
print(f"True causal effect: {true_causal_effect}")
print("\nFirst 5 SNPs:")
data.head()

## PyMR Analysis

### 1. Inverse Variance Weighted (IVW)

In [ ]:
# IVW analysis
ivw_result = ivw(
    beta_exp=data['beta_exp'].values,
    se_exp=data['se_exp'].values,
    beta_out=data['beta_out'].values,
    se_out=data['se_out'].values
)

print("IVW Results (PyMR):")
print(f"  Beta: {ivw_result['beta']:.4f}")
print(f"  SE: {ivw_result['se']:.4f}")
print(f"  P-value: {ivw_result['pval']:.4e}")
print(f"  OR: {ivw_result['OR']:.4f}")
print(f"  95% CI: [{ivw_result['OR_lci']:.4f}, {ivw_result['OR_uci']:.4f}]")
print(f"  N SNPs: {ivw_result['nsnp']}")

### 2. Weighted Median

In [ ]:
# Weighted Median analysis
wm_result = weighted_median(
    beta_exp=data['beta_exp'].values,
    se_exp=data['se_exp'].values,
    beta_out=data['beta_out'].values,
    se_out=data['se_out'].values,
    n_bootstrap=1000
)

print("Weighted Median Results (PyMR):")
print(f"  Beta: {wm_result['beta']:.4f}")
print(f"  SE: {wm_result['se']:.4f}")
print(f"  P-value: {wm_result['pval']:.4e}")
print(f"  OR: {wm_result['OR']:.4f}")
print(f"  95% CI: [{wm_result['OR_lci']:.4f}, {wm_result['OR_uci']:.4f}]")
print(f"  N SNPs: {wm_result['nsnp']}")

### 3. MR-Egger

In [ ]:
# MR-Egger analysis
egger_result = mr_egger(
    beta_exp=data['beta_exp'].values,
    se_exp=data['se_exp'].values,
    beta_out=data['beta_out'].values,
    se_out=data['se_out'].values
)

print("MR-Egger Results (PyMR):")
print(f"  Slope (Beta): {egger_result['beta']:.4f}")
print(f"  Slope SE: {egger_result['se']:.4f}")
print(f"  Slope P-value: {egger_result['pval']:.4e}")
print(f"  Intercept: {egger_result['intercept']:.4f}")
print(f"  Intercept SE: {egger_result['intercept_se']:.4f}")
print(f"  Intercept P-value: {egger_result['intercept_pval']:.4e}")
print(f"  N SNPs: {egger_result['nsnp']}")

### 4. Heterogeneity Statistics

In [ ]:
# Cochran's Q test for IVW
q_result = cochrans_q(
    beta_exp=data['beta_exp'].values,
    se_exp=data['se_exp'].values,
    beta_out=data['beta_out'].values,
    se_out=data['se_out'].values,
    causal_estimate=ivw_result['beta']
)

print("Heterogeneity Statistics (PyMR):")
print(f"  Cochran's Q: {q_result['Q']:.4f}")
print(f"  Q df: {q_result['Q_df']}")
print(f"  Q P-value: {q_result['Q_pval']:.4e}")
print(f"  I²: {q_result['I2']:.2f}%")

## Summary Table

Summary of all PyMR results for comparison with TwoSampleMR:

In [ ]:
# Create summary table
summary = pd.DataFrame([
    {
        'Method': 'IVW',
        'Beta': ivw_result['beta'],
        'SE': ivw_result['se'],
        'P-value': ivw_result['pval'],
        'N SNPs': ivw_result['nsnp']
    },
    {
        'Method': 'Weighted Median',
        'Beta': wm_result['beta'],
        'SE': wm_result['se'],
        'P-value': wm_result['pval'],
        'N SNPs': wm_result['nsnp']
    },
    {
        'Method': 'MR-Egger (slope)',
        'Beta': egger_result['beta'],
        'SE': egger_result['se'],
        'P-value': egger_result['pval'],
        'N SNPs': egger_result['nsnp']
    },
    {
        'Method': 'MR-Egger (intercept)',
        'Beta': egger_result['intercept'],
        'SE': egger_result['intercept_se'],
        'P-value': egger_result['intercept_pval'],
        'N SNPs': egger_result['nsnp']
    }
])

print("\nPyMR Results Summary:")
summary

## Equivalent TwoSampleMR Code (R)

Below is the equivalent R code using TwoSampleMR. Users can run this code to verify PyMR results.

### Installation

```r
# Install TwoSampleMR if not already installed
if (!require("remotes")) {
    install.packages("remotes")
}
remotes::install_github("MRCIEU/TwoSampleMR")
```

### Data Preparation

First, save the Python data to CSV:

In [ ]:
# Save data for R comparison
data.to_csv('/tmp/pymr_validation_data.csv', index=False)
print("Data saved to /tmp/pymr_validation_data.csv")
print("You can now load this in R using the code below.")

### R Code for TwoSampleMR Analysis

```r
# Load required library
library(TwoSampleMR)

# Read the data
data <- read.csv("/tmp/pymr_validation_data.csv")

# Format exposure data
exposure_dat <- format_data(
    data,
    type = "exposure",
    snp_col = "SNP",
    beta_col = "beta_exp",
    se_col = "se_exp",
    effect_allele_col = "A1",  # You may need to add these columns
    other_allele_col = "A2",
    pval_col = "pval_exp"       # You may need to calculate these
)

# Format outcome data
outcome_dat <- format_data(
    data,
    type = "outcome",
    snp_col = "SNP",
    beta_col = "beta_out",
    se_col = "se_out",
    effect_allele_col = "A1",
    other_allele_col = "A2",
    pval_col = "pval_out"
)

# Harmonize data
harmonised_dat <- harmonise_data(
    exposure_dat = exposure_dat,
    outcome_dat = outcome_dat
)

# Run MR analysis
mr_results <- mr(harmonised_dat, method_list = c(
    "mr_ivw",
    "mr_weighted_median",
    "mr_egger_regression"
))

print("TwoSampleMR Results:")
print(mr_results)

# Heterogeneity statistics
hetero <- mr_heterogeneity(harmonised_dat)
print("\nHeterogeneity Statistics:")
print(hetero)

# Pleiotropy test (MR-Egger intercept)
pleio <- mr_pleiotropy_test(harmonised_dat)
print("\nPleiotropy Test (MR-Egger Intercept):")
print(pleio)
```

## Expected TwoSampleMR Results

Based on our PyMR implementation and the same data, TwoSampleMR should produce results that match within numerical precision (typically within 0.1% for point estimates).

### Expected Correspondence:

| Metric | PyMR Output | TwoSampleMR Column | Expected Match |
|--------|-------------|-------------------|----------------|
| **IVW Beta** | `ivw_result['beta']` | `mr_results$b` (where method="Inverse variance weighted") | ≈ exact |
| **IVW SE** | `ivw_result['se']` | `mr_results$se` | ≈ exact |
| **IVW P-value** | `ivw_result['pval']` | `mr_results$pval` | ≈ exact |
| **Weighted Median Beta** | `wm_result['beta']` | `mr_results$b` (where method="Weighted median") | ± 5%* |
| **Weighted Median SE** | `wm_result['se']` | `mr_results$se` | ± 10%* |
| **MR-Egger Slope** | `egger_result['beta']` | `mr_results$b` (where method="MR Egger") | ≈ exact |
| **MR-Egger Intercept** | `egger_result['intercept']` | `pleio$egger_intercept` | ≈ exact |
| **Cochran's Q** | `q_result['Q']` | `hetero$Q` (where method="Inverse variance weighted") | ≈ exact |
| **I²** | `q_result['I2']` | Not directly reported (calculate as (Q-df)/Q*100) | ≈ exact |

*Note: Weighted median uses bootstrap resampling, so results may vary slightly due to random sampling. Using the same random seed (42) should give very similar results.

## Simplified R Code (Using Raw Data)

For a more direct comparison without the formatting overhead, you can use the raw calculation approach:

```r
# Load data
data <- read.csv("/tmp/pymr_validation_data.csv")

# Extract vectors
beta_exp <- data$beta_exp
se_exp <- data$se_exp
beta_out <- data$beta_out
se_out <- data$se_out

# Manual IVW calculation (to verify)
wald_ratio <- beta_out / beta_exp
wald_se <- abs(se_out / beta_exp)
weights <- 1 / wald_se^2
ivw_beta <- sum(wald_ratio * weights) / sum(weights)
ivw_se <- sqrt(1 / sum(weights))
ivw_pval <- 2 * pnorm(-abs(ivw_beta / ivw_se))

cat("Manual IVW calculation:\n")
cat(sprintf("  Beta: %.4f\n", ivw_beta))
cat(sprintf("  SE: %.4f\n", ivw_se))
cat(sprintf("  P-value: %.4e\n", ivw_pval))

# This should match PyMR exactly
```

## Validation Checklist

When running the comparison, verify:

- [ ] IVW estimates match within 0.1%
- [ ] IVW standard errors match within 0.1%
- [ ] IVW p-values match within 1%
- [ ] Weighted median estimates are similar (within 5% - due to bootstrap)
- [ ] MR-Egger slope matches within 0.1%
- [ ] MR-Egger intercept matches within 0.1%
- [ ] Cochran's Q statistic matches within 0.1%
- [ ] Heterogeneity I² matches within 1%

## Notes on Differences

Minor differences may occur due to:

1. **Bootstrap Methods**: Weighted median uses random resampling. Even with the same seed, different RNG implementations (NumPy vs R) may produce slightly different results.

2. **Numerical Precision**: Different linear algebra libraries (NumPy/SciPy vs R base) may have minor floating-point differences.

3. **Edge Cases**: Treatment of edge cases (e.g., when SE approaches zero) may differ slightly.

These differences should be negligible for practical purposes (< 1% for point estimates).

## Conclusion

PyMR implements MR methods following the same statistical principles as TwoSampleMR. The results should match within numerical precision for:

- Deterministic methods (IVW, MR-Egger)
- Heterogeneity statistics (Cochran's Q, I²)

And should be very similar (within a few percent) for:

- Stochastic methods (Weighted Median - due to bootstrap)

Users are encouraged to run this validation with their own datasets to verify PyMR's accuracy.